# Sample 01: OpenAI SDK വഴി വേഗത്തിലുള്ള ചാറ്റ്

ഈ നോട്ട്‌ബുക്ക് Microsoft Foundry Local ഉപയോഗിച്ച് OpenAI SDK എങ്ങനെ ഉപയോഗിക്കാമെന്ന് വേഗത്തിലുള്ള ചാറ്റ് ഇടപെടലുകൾക്കായി കാണിക്കുന്നു.

## അവലോകനം

ഈ ഉദാഹരണം കാണിക്കുന്നു:
- Foundry Local ഉപയോഗിച്ച് OpenAI Python SDK ഉപയോഗിക്കൽ
- Azure OpenAIയും ലോക്കൽ Foundry കോൺഫിഗറേഷനുകളും കൈകാര്യം ചെയ്യൽ
- ശരിയായ പിശക് കൈകാര്യം ചെയ്യലും ഫാൾബാക്ക് തന്ത്രങ്ങളും നടപ്പിലാക്കൽ
- സർവീസ് മാനേജ്മെന്റിനായി FoundryLocalManager ഉപയോഗിക്കൽ


## മുൻ‌വശം ആവശ്യങ്ങൾ

നിങ്ങൾക്കുണ്ടെന്ന് ഉറപ്പാക്കുക:
1. Foundry Local ഇൻസ്റ്റാൾ ചെയ്ത് പ്രവർത്തനക്ഷമമാക്കിയിട്ടുണ്ട്
2. ഒരു മോഡൽ ലോഡ് ചെയ്തിട്ടുണ്ട് (ഉദാഹരണം: `foundry model run phi-4-mini`)
3. ആവശ്യമായ Python പാക്കേജുകൾ ഇൻസ്റ്റാൾ ചെയ്തിട്ടുണ്ട്

### ആശ്രിതങ്ങൾ ഇൻസ്റ്റാൾ ചെയ്യുക


In [ ]:
# Install required packages if not already installed
!pip install openai foundry-local-sdk

## ആവശ്യമായ ലൈബ്രറികൾ ഇറക്കുമതി ചെയ്യുക


In [ ]:
import os
import sys
from openai import OpenAI

try:
    from foundry_local import FoundryLocalManager
    FOUNDRY_SDK_AVAILABLE = True
    print("✅ Foundry Local SDK is available")
except ImportError:
    FOUNDRY_SDK_AVAILABLE = False
    print("⚠️ Foundry Local SDK not available, will use manual configuration")

## ക്രമീകരണം

നിങ്ങളുടെ ക്രമീകരണം സജ്ജമാക്കുക. നിങ്ങൾക്ക് ഉപയോഗിക്കാം:
1. **Azure OpenAI** (ക്ലൗഡ് അടിസ്ഥാനമാക്കിയുള്ളത്)
2. **Foundry Local** (പ്രാദേശിക ഇൻഫറൻസ്)

### ഓപ്ഷൻ 1: Azure OpenAI ക്രമീകരണം


In [ ]:
# Azure OpenAI Configuration (uncomment and set your values)
# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource.openai.azure.com"
# os.environ["AZURE_OPENAI_API_KEY"] = "your-api-key"
# os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"
# os.environ["MODEL"] = "your-deployment-name"

### ഓപ്ഷൻ 2: ഫൗണ്ട്രി ലോക്കൽ കോൺഫിഗറേഷൻ


In [ ]:
# Foundry Local Configuration
MODEL_ALIAS = "phi-4-mini"  # Change this to your preferred model
BASE_URL = "http://localhost:8000"  # Default Foundry Local URL
API_KEY = ""  # Usually not needed for local

## ക്ലയന്റ് ആരംഭിക്കൽ

നിങ്ങളുടെ കോൺഫിഗറേഷനിന്റെ അടിസ്ഥാനത്തിൽ OpenAI ക്ലയന്റ് സൃഷ്ടിക്കുക:


In [ ]:
def create_client():
    """Create and configure OpenAI client."""
    
    # Check for Azure OpenAI configuration
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    azure_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
    azure_api_version = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-08-01-preview")
    
    if azure_endpoint and azure_api_key:
        # Azure OpenAI path
        model = os.environ.get("MODEL", "your-deployment-name")
        client = OpenAI(
            base_url=f"{azure_endpoint}/openai",
            api_key=azure_api_key,
            default_query={"api-version": azure_api_version},
        )
        print(f"🌐 Using Azure OpenAI with model: {model}")
        return client, model
    
    # Foundry Local path
    if FOUNDRY_SDK_AVAILABLE:
        try:
            # Use FoundryLocalManager for proper service management
            manager = FoundryLocalManager(MODEL_ALIAS)
            model_info = manager.get_model_info(MODEL_ALIAS)
            
            # Configure OpenAI client to use local Foundry service
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key  # API key is not required for local usage
            )
            model = model_info.id
            print(f"🏠 Using Foundry Local SDK with model: {model}")
            return client, model
        except Exception as e:
            print(f"⚠️ Could not use Foundry SDK ({e}), falling back to manual configuration")
    
    # Fallback to manual configuration
    client = OpenAI(
        base_url=f"{BASE_URL}/v1",
        api_key=API_KEY
    )
    model = MODEL_ALIAS
    print(f"🔧 Using manual configuration with model: {model}")
    return client, model

# Initialize the client
client, model = create_client()

## അടിസ്ഥാന ചാറ്റ് ഉദാഹരണം

ഒരു ലളിതമായ ചാറ്റ് ഇടപെടൽ പരീക്ഷിക്കാം:


In [ ]:
def chat_with_model(prompt, max_tokens=128):
    """Send a chat message to the model and get a response."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens
        )
        
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# Test with a simple prompt
prompt = "Say hello from Foundry Local and explain what you are in one sentence."
print(f"👤 User: {prompt}")
print("\n🤖 Assistant:")
response = chat_with_model(prompt)
print(response)

## ഇന്ററാക്ടീവ് ചാറ്റ് സെഷൻ

മോഡൽ എങ്ങനെ പ്രതികരിക്കുന്നു എന്ന് കാണാൻ വ്യത്യസ്ത തരത്തിലുള്ള പ്രോംപ്റ്റുകൾ പരീക്ഷിക്കുക:


In [ ]:
# Example prompts to try
example_prompts = [
    "What are the benefits of edge AI?",
    "Explain the difference between local and cloud AI inference.",
    "Write a short Python function to calculate the factorial of a number.",
    "What is Microsoft Foundry Local?"
]

for i, prompt in enumerate(example_prompts, 1):
    print(f"\n{'='*50}")
    print(f"Example {i}: {prompt}")
    print(f"{'='*50}")
    
    response = chat_with_model(prompt, max_tokens=200)
    print(response)

## Advanced Usage: Streaming Responses

For longer responses, streaming can provide a better user experience:


In [ ]:
def chat_with_streaming(prompt, max_tokens=300):
    """Send a chat message with streaming response."""
    try:
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            stream=True
        )
        
        print("🤖 Assistant (streaming):")
        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                print(content, end="", flush=True)
                full_response += content
        print("\n")  # New line after streaming
        return full_response
    except Exception as e:
        return f"Error: {e}"

# Test streaming with a longer prompt
streaming_prompt = "Explain the advantages of running AI models locally on device versus in the cloud. Include privacy, latency, and cost considerations."
print(f"👤 User: {streaming_prompt}\n")
streaming_response = chat_with_streaming(streaming_prompt)

## പിശക് കൈകാര്യം ചെയ്യലും ഡയഗ്നോസ്റ്റിക്സും

സേവന നിലയും ലഭ്യമായ മോഡലുകളും പരിശോധിക്കാം:


In [ ]:
def check_service_health():
    """Check Foundry Local service health."""
    try:
        # Try to list available models
        models_response = client.models.list()
        models = [model.id for model in models_response.data]
        
        print("✅ Service is healthy")
        print(f"📋 Available models: {models}")
        print(f"🎯 Current model: {model}")
        
        if model in models:
            print("✅ Current model is available")
        else:
            print("⚠️ Current model may not be loaded")
            
    except Exception as e:
        print(f"❌ Service check failed: {e}")
        print("\n🔧 Troubleshooting tips:")
        print("1. Make sure Foundry Local is running")
        print("2. Check if a model is loaded: foundry service ps")
        print("3. Verify the endpoint URL is correct")

check_service_health()

## കസ്റ്റം പ്രോംപ്റ്റ് ടെസ്റ്റിംഗ്

താഴെയുള്ള സെൽ ഉപയോഗിച്ച് നിങ്ങളുടെ സ്വന്തം പ്രോംപ്റ്റുകൾ പരീക്ഷിക്കുക:


In [ ]:
# Enter your custom prompt here
custom_prompt = "Write a haiku about artificial intelligence running on edge devices."

print(f"👤 User: {custom_prompt}\n")
custom_response = chat_with_model(custom_prompt, max_tokens=100)
print(f"🤖 Assistant: {custom_response}")

## സംഗ്രഹം

ഈ നോട്ട്‌ബുക്ക് പ്രദർശിപ്പിച്ചത്:

1. **✅ ക്ലയന്റ് സെറ്റപ്പ്**: Foundry Local ഉപയോഗിച്ച് OpenAI SDK എങ്ങനെ കോൺഫിഗർ ചെയ്യാം
2. **✅ അടിസ്ഥാന ചാറ്റ്**: ലളിതമായ അഭ്യർത്ഥന-പ്രതികരണ ഇടപെടലുകൾ
3. **✅ സ്ട്രീമിംഗ്**: യഥാർത്ഥ സമയ പ്രതികരണ സൃഷ്ടി
4. **✅ പിശക് കൈകാര്യം ചെയ്യൽ**: ശക്തമായ പിശക് കൈകാര്യം ചെയ്യലും ഡയഗ്നോസ്റ്റിക്സും
5. **✅ സർവീസ് ഹെൽത്ത്**: മോഡൽ ലഭ്യതയും സർവീസ് നിലയും പരിശോധിക്കൽ

### അടുത്ത ഘട്ടങ്ങൾ

- **സാമ്പിൾ 02**: Azure OpenAI പിന്തുണയോടെയുള്ള പുരോഗമിച്ച SDK സംയോജനം
- **സാമ്പിൾ 04**: Chainlit ചാറ്റ് അപ്ലിക്കേഷൻ നിർമ്മാണം
- **സാമ്പിൾ 05**: മൾട്ടി-ഏജന്റ് ഓർക്കസ്ട്രേഷൻ സിസ്റ്റങ്ങൾ
- **സാമ്പിൾ 06**: ബുദ്ധിമുട്ടുള്ള മോഡൽ റൂട്ടിംഗ്

### Foundry Local ന്റെ പ്രധാന ഗുണങ്ങൾ

- 🔒 **സ്വകാര്യത**: ഡാറ്റ നിങ്ങളുടെ ഉപകരണത്തിൽ നിന്ന് ഒരിക്കലും പുറത്തുപോകില്ല
- ⚡ **വേഗം**: കുറഞ്ഞ ലാറ്റൻസി ഉള്ള ലോക്കൽ ഇൻഫറൻസ്
- 💰 **ചെലവ്**: API ഉപയോഗ ചെലവുകൾ ഇല്ല
- 🔌 **ഓഫ്‌ലൈൻ**: ഇന്റർനെറ്റ് കണക്ഷൻ ഇല്ലാതെ പ്രവർത്തിക്കുന്നു
- 🛠️ **സമർത്ഥ്യത**: OpenAI-സമർത്ഥമായ API


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**അസൂയാപത്രം**:  
ഈ രേഖ AI വിവർത്തന സേവനം [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് വിവർത്തനം ചെയ്തതാണ്. നാം കൃത്യതയ്ക്ക് ശ്രമിച്ചിട്ടുണ്ടെങ്കിലും, സ്വയം പ്രവർത്തിക്കുന്ന വിവർത്തനങ്ങളിൽ പിശകുകൾ അല്ലെങ്കിൽ തെറ്റുകൾ ഉണ്ടാകാമെന്ന് ദയവായി ശ്രദ്ധിക്കുക. അതിന്റെ മാതൃഭാഷയിലുള്ള യഥാർത്ഥ രേഖ അധികാരപരമായ ഉറവിടമായി കണക്കാക്കപ്പെടണം. നിർണായക വിവരങ്ങൾക്ക്, പ്രൊഫഷണൽ മനുഷ്യ വിവർത്തനം ശുപാർശ ചെയ്യപ്പെടുന്നു. ഈ വിവർത്തനം ഉപയോഗിക്കുന്നതിൽ നിന്നുണ്ടാകുന്ന ഏതെങ്കിലും തെറ്റിദ്ധാരണകൾക്കോ തെറ്റായ വ്യാഖ്യാനങ്ങൾക്കോ ഞങ്ങൾ ഉത്തരവാദികളല്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
